In [1]:
!pip install denoiser onnx onnxruntime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.8/49.8 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 40.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.1/72.1 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 42.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 32.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from IPython import display as disp
import torch
import torchaudio
from denoiser import pretrained
from denoiser.dsp import convert_audio

In [3]:
!wget https://facebookresearch.github.io/denoiser/audio/noisy/alex_noisy.mp3
!wget https://github.com/MohammadRaziei/libspeech/releases/download/resources/audio-fa-noisy.mp3

--2025-03-25 10:51:42--  https://facebookresearch.github.io/denoiser/audio/noisy/alex_noisy.mp3
Resolving facebookresearch.github.io (facebookresearch.github.io)... 185.199.109.153, 185.199.111.153, 185.199.108.153, ...
Connecting to facebookresearch.github.io (facebookresearch.github.io)|185.199.109.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 339840 (332K) [audio/mp3]
Saving to: ‘alex_noisy.mp3’

alex_noisy.mp3      100%[===================>] 331.88K  --.-KB/s    in 0.06s   

2025-03-25 10:51:43 (5.28 MB/s) - ‘alex_noisy.mp3’ saved [339840/339840]

--2025-03-25 10:51:43--  https://github.com/MohammadRaziei/libspeech/releases/download/resources/audio-fa-noisy.mp3
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://objects.githubusercontent.com/github-production-release-asset-2e65be/948422393/1bdc893f-db93-412d-9fbb-e52

In [ ]:
model = pretrained.dns64()
wav, sr = torchaudio.load('audio-fa-noisy.mp3')
wav = wav[:,:sr*10]
wav = convert_audio(wav, sr, model.sample_rate, model.chin)
with torch.no_grad():
    denoised = model(wav[None])[0]
disp.display(disp.Audio(wav.data.cpu().numpy(), rate=model.sample_rate))
disp.display(disp.Audio(denoised.data.cpu().numpy(), rate=model.sample_rate))

In [ ]:
import torch

model = pretrained.dns64().cuda()

dummy_input = torch.randn(1, model.chin, 160000).cuda()  # [batch_size, channels, length]

onnx_file_path = "dns64_model.onnx"
with torch.no_grad():
    torch.onnx.export(
        model,
        dummy_input,
        onnx_file_path,
        export_params=True,
        opset_version=13,
        do_constant_folding=True,
        input_names=["input"],
        output_names=["output"],
        dynamic_axes={
            "input": {0: "batch_size", 2: "length"},
            "output": {0: "batch_size", 2: "length"}
        }
    )

print(f"Model saved to {onnx_file_path}")

In [ ]:
import torch

model = pretrained.dns48().cuda()

dummy_input = torch.randn(1, model.chin, 160000).cuda()  # [batch_size, channels, length]

onnx_file_path = "dns48_model.onnx"
with torch.no_grad():
    torch.onnx.export(
        model,
        dummy_input,
        onnx_file_path,
        export_params=True,
        opset_version=13,
        do_constant_folding=True,
        input_names=["input"],
        output_names=["output"],
        dynamic_axes={
            "input": {0: "batch_size", 2: "length"},
            "output": {0: "batch_size", 2: "length"}
        }
    )

print(f"Model saved to {onnx_file_path}")

In [ ]:
model = pretrained.master64().cuda()

dummy_input = torch.randn(1, model.chin, 160000).cuda()  # [batch_size, channels, length]

onnx_file_path = "master64_model.onnx"
with torch.no_grad():
    torch.onnx.export(
        model,
        dummy_input,
        onnx_file_path,
        export_params=True,
        opset_version=13,
        do_constant_folding=True,
        input_names=["input"],
        output_names=["output"],
        dynamic_axes={
            "input": {0: "batch_size", 2: "length"},
            "output": {0: "batch_size", 2: "length"}
        }
    )

print(f"Model saved to {onnx_file_path}")

In [ ]:
model = pretrained.valentini_nc().cuda()

dummy_input = torch.randn(1, model.chin, 160000).cuda()  # [batch_size, channels, length]

onnx_file_path = "valentini_nc_model.onnx"
with torch.no_grad():
    torch.onnx.export(
        model,
        dummy_input,
        onnx_file_path,
        export_params=True,
        opset_version=13,
        do_constant_folding=True,
        input_names=["input"],
        output_names=["output"],
        dynamic_axes={
            "input": {0: "batch_size", 2: "length"},
            "output": {0: "batch_size", 2: "length"}
        }
    )

print(f"Model saved to {onnx_file_path}")